# MST-GNN model - dynamic approach - Stock Market

## Import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.loader import DataLoader

# Import your custom modules
# If these fail, ensure the .py files are in the same folder as this notebook
from models.MST import MST_GNN
from dataset import InMemoryDynamicSP100
from models.train import train

# Setup Device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using Device: {DEVICE}")

## Setting hyper parameters

In [ ]:
# Data Configuration
PAST_WINDOW = 30       # Look-back window (Delta t)
FUTURE_WINDOW = 1      # Prediction horizon
BATCH_SIZE = 32

# MSCMG Graph Generation Params
GRAPH_RETURN_THRESH = 0.01  # 1% movement threshold (increased from 0.0009 - was TOO sparse)
LABEL_RETURN_THRESH = 0.01  # 1% movement threshold for labels
MAX_DIST = 5           # Max Manhattan distance for edges (reduced from 8 to be more selective)

# Model Configuration
INPUT_FEATURES = 9     # Check your values.csv (e.g., Open, Close, High, Low...)
HIDDEN_SIZE = 64
GRAPH_LAYERS = 3       # 'M' layers
CROSS_LAYERS = 2       # 'C' layers
LEARNING_RATE = 0.001  # Increased from 0.0005 to speed up convergence

NUM_EPOCHS = 100

## Loading data

In [ ]:
print("Initializing Dataset...")

# We set force_process=True to regenerate with new 3-class labels
# After first run, you can set it back to False to load from disk
dataset = InMemoryDynamicSP100(
    root="./data", 
    past_window=PAST_WINDOW, 
    future_window=FUTURE_WINDOW,
    graph_return_threshold=GRAPH_RETURN_THRESH,
    label_return_threshold=LABEL_RETURN_THRESH,
    max_dist_threshold=MAX_DIST,
    force_process=True  # Set to True to regenerate with 3-class labels
)

print(f"Total Samples: {len(dataset)}")
print(f"Feature Shape (Node, Time, Feat): {dataset[0].x.shape}")
print(f"Sample Label (should be 0, 1, or 2): {dataset[0].y}")